# Guia Practica
Nombre: Adriana Cornejo Ulloa

# -- Sección 1: Contexto del Análisis --


"""
# Análisis de Compras Públicas en Ecuador

Este notebook presenta un análisis básico de los datos abiertos sobre procesos de contratación pública en Ecuador, usando la API oficial basada en OCDS (Open Contracting Data Standard).

El objetivo es entender patrones y montos contratados según año, región y tipo de contratación.
"""

# -- Sección 2: Cómo funciona la API --


"""
## Uso de la API de Contrataciones Abiertas (OCDS)

La API permite buscar procesos de contratación pública mediante:

- Palabras clave (`search`)
- Año (`year`)
- Página de resultados (`page`)
- Opcionalmente, comprador (`buyer`) y proveedor (`supplier`)

El endpoint principal para búsqueda es:

`https://datosabiertos.compraspublicas.gob.ec/PLATAFORMA/api/search_ocds`

Ejemplo de URL para buscar contratos del año 2020 con palabra clave "agua":

`https://datosabiertos.compraspublicas.gob.ec/PLATAFORMA/api/search_ocds?year=2020&search=agua&page=1`

La respuesta es un JSON con datos relevantes que luego procesamos con Python.
"""

# -- Sección 3: Código para cargar y explorar datos --



import requests
import pandas as pd
import plotly.express as px

# Parámetros para consulta
year = 2023
search_keyword = "Azuay"
page = 1

# Construcción de URL y parámetros
url = "https://datosabiertos.compraspublicas.gob.ec/PLATAFORMA/api/search_ocds"
params = {
    "year": year,
    "search": search_keyword,
    "page": page
}

# Solicitud a la API
response = requests.get(url, params=params)
data = response.json()

# Mostrar la estructura de la respuesta
print(f"Total registros encontrados: {data.get('total', 0)}")
print(f"Páginas totales: {data.get('pages', 0)}")

# Crear DataFrame con los datos si existen
if data.get("data"):
    df = pd.DataFrame(data["data"])
    display(df.head())
else:
    print("No se encontraron datos para los filtros seleccionados.")


# -- Sección 4: Visualizaciones básicas --



if data.get("data"):
    # Convertir fecha a datetime
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['month'] = df['date'].dt.month
    
    # Contratos por mes
    contratos_mes = df.groupby('month').size().reset_index(name='cantidad')
    
    # Visualización con Plotly
    fig = px.bar(contratos_mes, x='month', y='cantidad',
                 labels={'month': 'Mes', 'cantidad': 'Cantidad de Contratos'},
                 title=f'Cantidad de contratos en {year} para "{search_keyword}"')
    fig.show()


# -- Sección 5: Pequeñas conclusiones --


"""
- Para el año seleccionado y búsqueda por palabra clave, se observa la distribución mensual de contratos.
- La API permite filtrar y obtener datos dinámicos para diferentes análisis.
- La cantidad de contratos puede variar por mes, indicando posibles patrones estacionales o administrativos.
"""